This notebook contains the code to apply `bayes-drt` to the example spectra presented in the manuscript. The code to apply the Gaussian Process and Elastic Net DRT models to the same data can be found in the `comparisons` directory.

In [2]:
import numpy as np
import pandas as pd
import os 
import sys
import glob
import time
import matplotlib.pyplot as plt

if '../bayes-drt' not in sys.path:
    sys.path.append('../bayes-drt')
import drt
from stan_models import save_pickle,load_pickle
import eis_utils as gt

%load_ext autoreload
%autoreload 2

# Simulated data

In [3]:
dr = drt.DRT()
files = glob.glob('../data/simulated/Z_*.csv')
files = [f for f in files if f.find('trunc')==-1]
# fit one file to generate A matrices
df = pd.read_csv(files[0])
Z = df['Zreal'] + 1j*df['Zimag']
dr.ridge_fit(df['Freq'].values,Z.values)

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
../bayes-drt\drt.py:1049: RuntimeWarning: overflow encountered in exp
  return phi(y,epsilon)/(1+np.exp(2*(y+np.log(w_n*t_m))))
../bayes-drt\drt.py:327: UserWarning: Hyperparametric solution did not converge within 20 iterations
  warnings.warn(f'Hyperparametric solution did not converge within {max_iter} iterations')


In [802]:
"MAP fits"
# tau for plotting
tau_plot = np.logspace(-7,2,200)
start = time.time()

for file in files:
    print('-------------------------------')
    print(file)
    print('-------------------------------')
    suffix = file[file.find('_'):]
    Zoutfile = os.path.join('map_results',f'Zout{suffix}')
    Goutfile = os.path.join('map_results',f'Gout{suffix}')
    pkl = os.path.join('map_results','obj{}.pkl'.format(suffix[:-4]))
    if file.split('_')[2]=='noiseless.csv':
        sigma_min=0.005
    else:
        sigma_min=0.002
    if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
        print('Already ran')
    else:
        df = pd.read_csv(file)
        Z = df['Zreal'].values + 1j*df['Zimag'].values
        # fit
        file_start = time.time()
        dr.map_fit(df['Freq'].values,Z,dZ=False,init_from_ridge=False,sigma_min=sigma_min)
        print('File fit time: {:.2f} seconds'.format(time.time()-file_start))
        
        # save results
        Z_pred = dr.predict(df['Freq'].values)
        Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag]).T,columns=['freq','Zreal','Zimag'])
        Z_res['sigma_re'] = dr._opt_result['sigma_tot'][:len(df)]*dr._Z_scale
        Z_res['sigma_im'] = dr._opt_result['sigma_tot'][len(df):]*dr._Z_scale
        
        g_pred = dr.drt(eval_tau=tau_plot)
        g_res = pd.DataFrame(np.array([tau_plot,g_pred]).T,
                             columns=['tau','gamma'])
        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)
        
        # pickle the drt object for access to all other attributes
        save_pickle(dr,pkl)
        
print('================================')        
print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../data/simulated\Z_2RC_Macdonald_0.25.csv
-------------------------------
File fit time: 12.71 seconds


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


Dumped pickle to map_results\obj_2RC_Macdonald_0.25.pkl
-------------------------------
../data/simulated\Z_2RC_Macdonald_1.0.csv
-------------------------------
File fit time: 6.97 seconds
Dumped pickle to map_results\obj_2RC_Macdonald_1.0.pkl
-------------------------------
../data/simulated\Z_2RC_Macdonald_2.5.csv
-------------------------------
File fit time: 3.50 seconds
Dumped pickle to map_results\obj_2RC_Macdonald_2.5.pkl
-------------------------------
../data/simulated\Z_2RC_noiseless.csv
-------------------------------
Already ran
-------------------------------
../data/simulated\Z_2RC_Orazem_0.25.csv
-------------------------------
File fit time: 17.38 seconds
Dumped pickle to map_results\obj_2RC_Orazem_0.25.pkl
-------------------------------
../data/simulated\Z_2RC_Orazem_1.0.csv
-------------------------------
File fit time: 4.86 seconds
Dumped pickle to map_results\obj_2RC_Orazem_1.0.pkl
-------------------------------
../data/simulated\Z_2RC_Orazem_2.5.csv
------------

Dumped pickle to map_results\obj_ZARC-RL_Orazem_1.0.pkl
-------------------------------
../data/simulated\Z_ZARC-RL_Orazem_2.5.csv
-------------------------------
File fit time: 1.32 seconds
Dumped pickle to map_results\obj_ZARC-RL_Orazem_2.5.pkl
-------------------------------
../data/simulated\Z_ZARC-RL_uniform_0.25.csv
-------------------------------
File fit time: 4.56 seconds
Dumped pickle to map_results\obj_ZARC-RL_uniform_0.25.pkl
-------------------------------
../data/simulated\Z_ZARC-RL_uniform_1.0.csv
-------------------------------
File fit time: 2.10 seconds
Dumped pickle to map_results\obj_ZARC-RL_uniform_1.0.pkl
-------------------------------
../data/simulated\Z_ZARC-RL_uniform_2.5.csv
-------------------------------
File fit time: 2.01 seconds
Dumped pickle to map_results\obj_ZARC-RL_uniform_2.5.pkl
-------------------------------
../data/simulated\Z_ZARC_Macdonald_0.25.csv
-------------------------------
File fit time: 5.87 seconds
Dumped pickle to map_results\obj_ZAR

In [878]:
"MCMC sampling"
# tau for plotting
tau_plot = np.logspace(-7,2,200)
start = time.time()

for file in files:
    print('-------------------------------')
    print(file)
    print('-------------------------------')
    suffix = file[file.find('_'):]
    Zoutfile = os.path.join('bayes_results',f'Zout{suffix}')
    Goutfile = os.path.join('bayes_results',f'Gout{suffix}')
    pkl = os.path.join('bayes_results','obj{}.pkl'.format(suffix[:-4]))
    if file.split('_')[2]=='noiseless.csv':
        sigma_min=0.005
    else:
        sigma_min=0.002
        
    if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
        print('Already ran')
    else:
        df = pd.read_csv(file)
        Z = df['Zreal'].values + 1j*df['Zimag'].values
        
        # fit
        file_start = time.time()
        dr.bayes_fit(df['Freq'].values,Z,dZ=False,init_from_ridge=True,sigma_min=sigma_min)
        print('File fit time: {:.2f} seconds'.format(time.time()-file_start))
        
        Z_pred = dr.predict(df['Freq'].values)
        # Z credibility interval
        Z_lo = dr.predict(df['Freq'].values,percentile=2.5)
        Z_hi = dr.predict(df['Freq'].values,percentile=97.5)
        Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag,Z_lo,Z_hi]).T,
                             columns=['freq','Zreal','Zimag','Zlo','Zhi'])
        Z_res['sigma_re'] = np.mean(dr._sample_result['sigma_tot'],axis=0)[:len(df)]*dr._Z_scale
        Z_res['sigma_im'] = np.mean(dr._sample_result['sigma_tot'],axis=0)[len(df):]*dr._Z_scale
        
        g_pred = dr.drt(eval_tau=tau_plot)
        # drt credibility interval
        g_lo = dr.drt(eval_tau=tau_plot,percentile=2.5)
        g_hi = dr.drt(eval_tau=tau_plot,percentile=97.5)
        g_res = pd.DataFrame(np.array([tau_plot,g_pred,g_lo,g_hi]).T,
                             columns=['tau','gamma','gamma_lo','gamma_hi'])
        
        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)
        
        # pickle the drt object for access to all other attributes
        save_pickle(dr,pkl)
        
print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../data/simulated\Z_2RC_Macdonald_0.25.csv
-------------------------------


../bayes-drt\drt.py:311: UserWarning: Hyperparametric solution did not converge within 20 iterations
  warnings.warn(f'Hyperparametric solution did not converge within {max_iter} iterations')
To run all diagnostics call pystan.check_hmc_diagnostics(fit)


File fit time: 129.66 seconds


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


Dumped pickle to bayes_results\obj_2RC_Macdonald_0.25.pkl
-------------------------------
../data/simulated\Z_2RC_Macdonald_1.0.csv
-------------------------------


...

File fit time: 87.35 seconds
Dumped pickle to bayes_results\obj_ZARC_uniform_0.25.pkl
-------------------------------
../data/simulated\Z_ZARC_uniform_1.0.csv
-------------------------------


To run all diagnostics call pystan.check_hmc_diagnostics(fit)


File fit time: 62.89 seconds
Dumped pickle to bayes_results\obj_ZARC_uniform_1.0.pkl
-------------------------------
../data/simulated\Z_ZARC_uniform_2.5.csv
-------------------------------


To run all diagnostics call pystan.check_hmc_diagnostics(fit)


File fit time: 65.77 seconds
Dumped pickle to bayes_results\obj_ZARC_uniform_2.5.pkl
Total fit time: 82.97 minutes


In [14]:
"Fit example file with 4 chains, 1000 iterations for comparison"
file = '../data/simulated/Z_2RC_uniform_0.25.csv'
df = pd.read_csv(file)
Z = df['Zreal'].values + 1j*df['Zimag'].values
suffix = file[file.find('_'):-4]
Zoutfile = os.path.join('bayes_results',f'Zout{suffix}_4x1000.csv')
Goutfile = os.path.join('bayes_results',f'Gout{suffix}_4x1000.csv')
pkl = os.path.join('bayes_results',f'obj{suffix}_4x1000.pkl')
tau_plot = np.logspace(-7,2,200)

file_start = time.time()
dr.bayes_fit(df['Freq'].values,Z,dZ=False,init_from_ridge=True,warmup=500,sample=500,chains=4)
print('File fit time: {:.2f} seconds'.format(time.time()-file_start))
        
Z_pred = dr.predict(df['Freq'].values)
# Z credibility interval
Z_lo = dr.predict(df['Freq'].values,percentile=2.5)
Z_hi = dr.predict(df['Freq'].values,percentile=97.5)
Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag,Z_lo,Z_hi]).T,
                     columns=['freq','Zreal','Zimag','Zlo','Zhi'])
Z_res['sigma_re'] = np.mean(dr._sample_result['sigma_tot'],axis=0)[:len(df)]*dr._Z_scale
Z_res['sigma_im'] = np.mean(dr._sample_result['sigma_tot'],axis=0)[len(df):]*dr._Z_scale

g_pred = dr.drt(eval_tau=tau_plot)
# drt credibility interval
g_lo = dr.drt(eval_tau=tau_plot,percentile=2.5)
g_hi = dr.drt(eval_tau=tau_plot,percentile=97.5)
g_res = pd.DataFrame(np.array([tau_plot,g_pred,g_lo,g_hi]).T,
                     columns=['tau','gamma','gamma_lo','gamma_hi'])

Z_res.to_csv(Zoutfile,index=False)
g_res.to_csv(Goutfile,index=False)

# pickle the drt object for access to all other attributes
save_pickle(dr,pkl)

To run all diagnostics call pystan.check_hmc_diagnostics(fit)


File fit time: 509.13 seconds
Dumped pickle to bayes_results\obj_2RC_uniform_0.25_4x1000.pkl


## Truncated RL-ZARC-ZARC

In [2]:
dr_trunc = drt.DRT(basis_freq=np.logspace(6,-2,81))
trunc_files = glob.glob('../data/simulated/Z_trunc*.csv')

# fit one file to generate A matrices
df = pd.read_csv(trunc_files[0])
Z = df['Zreal'] + 1j*df['Zimag']
dr_trunc.ridge_fit(df['Freq'].values,Z.values)

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


NameError: name 'dr_trunc' is not defined

In [908]:
"MAP fits"
# tau for plotting
tau_plot = np.logspace(-7,2,200)
start = time.time()

for file in trunc_files:
    print('-------------------------------')
    print(file)
    print('-------------------------------')
    suffix = file[file.find('_'):]
    Zoutfile = os.path.join('map_results',f'Zout{suffix}')
    Goutfile = os.path.join('map_results',f'Gout{suffix}')
    pkl = os.path.join('map_results','obj{}.pkl'.format(suffix[:-4]))
    if file.split('_')[2]=='noiseless.csv':
        sigma_min=0.005
    else:
        sigma_min=0.002
    if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
        print('Already ran')
    else:
        df = pd.read_csv(file)
        Z = df['Zreal'].values + 1j*df['Zimag'].values
        # fit
        file_start = time.time()
        dr_trunc.map_fit(df['Freq'].values,Z,dZ=False,init_from_ridge=False,sigma_min=sigma_min)
        print('File fit time: {:.2f} seconds'.format(time.time()-file_start))
        
        # save results
        Z_pred = dr_trunc.predict(df['Freq'].values)
        Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag]).T,columns=['freq','Zreal','Zimag'])
        Z_res['sigma_re'] = dr_trunc._opt_result['sigma_tot'][:len(df)]*dr_trunc._Z_scale
        Z_res['sigma_im'] = dr_trunc._opt_result['sigma_tot'][len(df):]*dr_trunc._Z_scale
        
        g_pred = dr_trunc.drt(eval_tau=tau_plot)
        g_res = pd.DataFrame(np.array([tau_plot,g_pred]).T,
                             columns=['tau','gamma'])
        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)
        
        # pickle the drt object for access to all other attributes
        save_pickle(dr_trunc,pkl)
        
print('================================')        
print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../data/simulated\Z_trunc_Macdonald_0.25.csv
-------------------------------
File fit time: 8.98 seconds
Dumped pickle to map_results\obj_trunc_Macdonald_0.25.pkl
-------------------------------
../data/simulated\Z_trunc_Macdonald_1.0.csv
-------------------------------
File fit time: 2.17 seconds
Dumped pickle to map_results\obj_trunc_Macdonald_1.0.pkl
-------------------------------
../data/simulated\Z_trunc_noiseless.csv
-------------------------------
File fit time: 10.80 seconds
Dumped pickle to map_results\obj_trunc_noiseless.pkl
Total fit time: 0.40 minutes


In [912]:
"MCMC sampling"
# tau for plotting
tau_plot = np.logspace(-7,2,200)
start = time.time()

for file in trunc_files:
    print('-------------------------------')
    print(file)
    print('-------------------------------')
    suffix = file[file.find('_'):]
    Zoutfile = os.path.join('bayes_results',f'Zout{suffix}')
    Goutfile = os.path.join('bayes_results',f'Gout{suffix}')
    pkl = os.path.join('bayes_results','obj{}.pkl'.format(suffix[:-4]))
    if file.split('_')[2]=='noiseless.csv':
        sigma_min=0.005
    else:
        sigma_min=0.002
        
    if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
        print('Already ran')
    else:
        df = pd.read_csv(file)
        Z = df['Zreal'].values + 1j*df['Zimag'].values
        
        # fit
        file_start = time.time()
        dr_trunc.bayes_fit(df['Freq'].values,Z,dZ=False,init_from_ridge=True,sigma_min=sigma_min)
        print('File fit time: {:.2f} seconds'.format(time.time()-file_start))
        
        Z_pred = dr_trunc.predict(df['Freq'].values)
        # Z credibility interval
        Z_lo = dr_trunc.predict(df['Freq'].values,percentile=2.5)
        Z_hi = dr_trunc.predict(df['Freq'].values,percentile=97.5)
        Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag,Z_lo,Z_hi]).T,
                             columns=['freq','Zreal','Zimag','Zlo','Zhi'])
        Z_res['sigma_re'] = np.mean(dr._sample_result['sigma_tot'],axis=0)[:len(df)]*dr._Z_scale
        Z_res['sigma_im'] = np.mean(dr._sample_result['sigma_tot'],axis=0)[len(df):]*dr._Z_scale
        
        g_pred = dr_trunc.drt(eval_tau=tau_plot)
        # drt credibility interval
        g_lo = dr_trunc.drt(eval_tau=tau_plot,percentile=2.5)
        g_hi = dr_trunc.drt(eval_tau=tau_plot,percentile=97.5)
        g_res = pd.DataFrame(np.array([tau_plot,g_pred,g_lo,g_hi]).T,
                             columns=['tau','gamma','gamma_lo','gamma_hi'])
        
        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)
        
        # pickle the drt object for access to all other attributes
        save_pickle(dr_trunc,pkl)
        
print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../data/simulated\Z_trunc_Macdonald_0.25.csv
-------------------------------


../bayes-drt\drt.py:311: UserWarning: Hyperparametric solution did not converge within 20 iterations
  warnings.warn(f'Hyperparametric solution did not converge within {max_iter} iterations')


File fit time: 114.40 seconds


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


Dumped pickle to bayes_results\obj_trunc_Macdonald_0.25.pkl
-------------------------------
../data/simulated\Z_trunc_Macdonald_1.0.csv
-------------------------------


File fit time: 61.44 seconds
Dumped pickle to bayes_results\obj_trunc_Macdonald_1.0.pkl
-------------------------------
../data/simulated\Z_trunc_noiseless.csv
-------------------------------


File fit time: 115.07 seconds
Dumped pickle to bayes_results\obj_trunc_noiseless.pkl
Total fit time: 4.88 minutes


# Experimental data
## LIB data from DRTtools

In [6]:
lib_files = ['../data/experimental/DRTtools_LIB_data.txt',
             '../data/experimental/DRTtools_LIB_data_qtr.csv'
            ]

dr_lib = drt.DRT(basis_freq=np.logspace(4,-5,101))

# run ridge_fit to generate A matrices
lib_df = pd.read_csv(lib_files[0],sep='\t',header=None)
lib_df = pd.DataFrame(lib_df.values,columns=['Freq','Zreal','Zimag'])
Z_lib = lib_df['Zreal'] + 1j*lib_df['Zimag']
f = lib_df['Freq'].values
Z_lib = Z_lib
dr_lib.ridge_fit(f,Z_lib.values)

# finer tau for plotting
tau_plot = np.logspace(np.log10(np.min(dr_lib.tau)),np.log10(np.max(dr_lib.tau)),200)

../bayes-drt\drt.py:1035: RuntimeWarning: overflow encountered in exp
  return phi(y,epsilon)/(1+np.exp(2*(y+np.log(w_n*t_m))))
../bayes-drt\drt.py:1117: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  A_main[n,:] = [quad(func,limits[0],limits[1],args=(w_n,t_m,epsilon),epsabs=1e-4)[0] for t_m in tau]
../bayes-drt\drt.py:311: UserWarning: Hyperparametric solution did not converge within 20 iterations
  warnings.warn(f'Hyperparametric solution did not converge within {max_iter} iterations')


In [7]:
"MAP fits"
start = time.time()

for file in lib_files:
    print('-------------------------------')
    print(file)
    print('-------------------------------')
    suffix = file[file.find('_'):]
    # strip file extension
    suffix = suffix[:suffix.find('.')]
    Zoutfile = os.path.join('map_results',f'Zout{suffix}.csv')
    Goutfile = os.path.join('map_results',f'Gout{suffix}.csv')
    pkl = os.path.join('map_results','obj{}.pkl'.format(suffix))
    if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
        print('Already ran')
    else:
        if file[-3:]=='txt':
            df = pd.read_csv(file,sep='\t',header=None)
            df = pd.DataFrame(df.values,columns=['Freq','Zreal','Zimag'])
        else:
            df = pd.read_csv(file)
        Z = df['Zreal'].values + 1j*df['Zimag'].values
        # fit
        file_start = time.time()
        # initializing from the ridge solution may be helpful for faster convergence for more complex spectra
        dr_lib.map_fit(df['Freq'].values,Z,dZ=False,init_from_ridge=True,max_iter=30e3)
        print('File fit time: {:.2f} seconds'.format(time.time()-file_start))
        # save results
        Z_pred = dr_lib.predict(df['Freq'].values)
        Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag]).T,columns=['freq','Zreal','Zimag'])
        g_pred = dr_lib.drt(eval_tau=tau_plot)
        g_res = pd.DataFrame(np.array([tau_plot,g_pred]).T,
                             columns=['tau','gamma'])
        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)
        
        # pickle the drt object for access to all other attributes
        save_pickle(dr_lib,pkl)
        
print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../data/experimental/DRTtools_LIB_data.txt
-------------------------------
File fit time: 23.43 seconds
Dumped pickle to map_results\obj_LIB_data.pkl
-------------------------------
../data/experimental/DRTtools_LIB_data_qtr.csv
-------------------------------
File fit time: 11.20 seconds
Dumped pickle to map_results\obj_LIB_data_qtr.pkl
Total fit time: 0.58 minutes


In [9]:
"MCMC sampling"
# tau for plotting
tau_plot = np.logspace(np.log10(np.min(dr_lib.tau)),np.log10(np.max(dr_lib.tau)),200)
start = time.time()

for file in lib_files:
    print('-------------------------------')
    print(file)
    print('-------------------------------')
    suffix = file[file.find('_'):]
    # strip file extension
    suffix = suffix[:suffix.find('.')]
    Zoutfile = os.path.join('bayes_results',f'Zout{suffix}.csv')
    Goutfile = os.path.join('bayes_results',f'Gout{suffix}.csv')
    pkl = os.path.join('bayes_results','obj{}.pkl'.format(suffix))
    if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
        print('Already ran')
    else:
        if file[-3:]=='txt':
            df = pd.read_csv(file,sep='\t',header=None)
            df = pd.DataFrame(df.values,columns=['Freq','Zreal','Zimag'])
        else:
            df = pd.read_csv(file)
        Z = df['Zreal'].values + 1j*df['Zimag'].values
        
        # fit
        file_start = time.time()
        # initializing from the ridge solution may be helpful for sampling for more complex spectra
        dr_lib.bayes_fit(df['Freq'].values,Z,dZ=False,init_from_ridge=True)
        print('File fit time: {:.2f} seconds'.format(time.time()-file_start))
        
        Z_pred = dr_lib.predict(df['Freq'].values)
        # Z credibility interval
        Z_lo = dr_lib.predict(df['Freq'].values,percentile=2.5)
        Z_hi = dr_lib.predict(df['Freq'].values,percentile=97.5)
        Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag,Z_lo,Z_hi]).T,
                             columns=['freq','Zreal','Zimag','Zlo','Zhi'])
        Z_res['sigma_re'] = np.mean(dr._sample_result['sigma_tot'],axis=0)[:len(df)]*dr._Z_scale
        Z_res['sigma_im'] = np.mean(dr._sample_result['sigma_tot'],axis=0)[len(df):]*dr._Z_scale
        
        g_pred = dr_lib.drt(eval_tau=tau_plot)
        # drt crediibility interval
        g_lo = dr_lib.drt(eval_tau=tau_plot,percentile=2.5)
        g_hi = dr_lib.drt(eval_tau=tau_plot,percentile=97.5)
        g_res = pd.DataFrame(np.array([tau_plot,g_pred,g_lo,g_hi]).T,
                             columns=['tau','gamma','gamma_lo','gamma_hi'])
        
        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)
        
        # pickle the drt object for access to all other attributes
        save_pickle(dr_lib,pkl)
        
print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
../data/experimental/DRTtools_LIB_data.txt
-------------------------------


To run all diagnostics call pystan.check_hmc_diagnostics(fit)


File fit time: 195.32 seconds


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)


Dumped pickle to bayes_results\obj_LIB_data.pkl
-------------------------------
../data/experimental/DRTtools_LIB_data_qtr.csv
-------------------------------


File fit time: 120.17 seconds
Dumped pickle to bayes_results\obj_LIB_data_qtr.pkl
Total fit time: 5.26 minutes


## Protonic ceramic microelectrode

In [14]:
tco_file = '../data/experimental/PDAC_COM3_02109_Contact10_2065C_500C.txt'
tco_df = gt.read_eis_zdata(tco_file)

dr_tco = drt.DRT(basis_freq=np.logspace(7,-3,101))

# run ridge_fit to generate A matrices
Z_tco = tco_df['Zreal'] + 1j*tco_df['Zimag']
f = tco_df['Freq'].values
dr_tco.ridge_fit(f,Z_tco.values)

# finer tau for plotting
tau_plot_tco = np.logspace(np.log10(np.min(dr_tco.tau)),np.log10(np.max(dr_tco.tau)),200)

In [17]:
"MAP fits"
start = time.time()
file = tco_file

for outliers in [True,False]:
    print('-------------------------------')
    print(f'Outliers={outliers}')
    print('-------------------------------')
    suffix = '_PDAC'
    if outliers:
        suffix += '_outliers'
    Zoutfile = os.path.join('map_results',f'Zout{suffix}.csv')
    Goutfile = os.path.join('map_results',f'Gout{suffix}.csv')
    pkl = os.path.join('map_results','obj{}.pkl'.format(suffix))
    if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
        print('Already ran')
    else:
        df = gt.read_eis_zdata(file)
        Z = df['Zreal'].values + 1j*df['Zimag'].values
        # fit
        file_start = time.time()
        # initializing from the ridge solution may be helpful for faster convergence for more complex spectra
        dr_tco.map_fit(df['Freq'].values,Z,dZ=False,init_from_ridge=True,max_iter=30e3,outliers=outliers)
        print('File fit time: {:.2f} seconds'.format(time.time()-file_start))
        # save results
        Z_pred = dr_tco.predict(df['Freq'].values)
        Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag]).T,columns=['freq','Zreal','Zimag'])
        g_pred = dr_tco.drt(eval_tau=tau_plot_tco)
        g_res = pd.DataFrame(np.array([tau_plot_tco,g_pred]).T,
                             columns=['tau','gamma'])
        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)
        
        # pickle the drt object for access to all other attributes
        save_pickle(dr_tco,pkl)
        
print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
Outliers=True
-------------------------------
File fit time: 28.22 seconds
Dumped pickle to map_results\obj_PDAC_outliers.pkl
-------------------------------
Outliers=False
-------------------------------
File fit time: 13.70 seconds
Dumped pickle to map_results\obj_PDAC.pkl
Total fit time: 0.70 minutes


In [28]:
"MCMC sampling"
# tau for plotting
start = time.time()

file = tco_file
for outliers in [True,False]:
    print('-------------------------------')
    print(f'Outliers={outliers}')
    print('-------------------------------')
    suffix = '_PDAC'
    if outliers:
        suffix += '_outliers'
        
    Zoutfile = os.path.join('bayes_results',f'Zout{suffix}.csv')
    Goutfile = os.path.join('bayes_results',f'Gout{suffix}.csv')
    pkl = os.path.join('bayes_results','obj{}.pkl'.format(suffix))
    
    if os.path.exists(Zoutfile) and os.path.exists(Goutfile):
        print('Already ran')
    else:
        df = gt.read_eis_zdata(file)
        Z = df['Zreal'].values + 1j*df['Zimag'].values

        # fit
        file_start = time.time()
        # initializing from the ridge solution may be helpful for sampling for more complex spectra
        dr_tco.bayes_fit(df['Freq'].values,Z,dZ=False,init_from_ridge=True,outliers=outliers)
        print('File fit time: {:.2f} seconds'.format(time.time()-file_start))

        Z_pred = dr_tco.predict(df['Freq'].values)
        # Z credibility interval
        Z_lo = dr_tco.predict(df['Freq'].values,percentile=2.5)
        Z_hi = dr_tco.predict(df['Freq'].values,percentile=97.5)
        Z_res = pd.DataFrame(np.array([df['Freq'],Z_pred.real,Z_pred.imag,Z_lo,Z_hi]).T,
                             columns=['freq','Zreal','Zimag','Zlo','Zhi'])
        Z_res['sigma_re'] = np.mean(dr._sample_result['sigma_tot'],axis=0)[:len(df)]*dr._Z_scale
        Z_res['sigma_im'] = np.mean(dr._sample_result['sigma_tot'],axis=0)[len(df):]*dr._Z_scale

        g_pred = dr_tco.drt(eval_tau=tau_plot_tco)
        # drt crediibility interval
        g_lo = dr_tco.drt(eval_tau=tau_plot_tco,percentile=2.5)
        g_hi = dr_tco.drt(eval_tau=tau_plot_tco,percentile=97.5)
        g_res = pd.DataFrame(np.array([tau_plot_tco,g_pred,g_lo,g_hi]).T,
                             columns=['tau','gamma','gamma_lo','gamma_hi'])

        Z_res.to_csv(Zoutfile,index=False)
        g_res.to_csv(Goutfile,index=False)

        # pickle the drt object for access to all other attributes
        save_pickle(dr_tco,pkl)
        
print('Total fit time: {:.2f} minutes'.format((time.time()-start)/60))

-------------------------------
Outliers=True
-------------------------------


../bayes-drt\drt.py:311: UserWarning: Hyperparametric solution did not converge within 20 iterations
  warnings.warn(f'Hyperparametric solution did not converge within {max_iter} iterations')
To run all diagnostics call pystan.check_hmc_diagnostics(fit)


File fit time: 222.17 seconds
Dumped pickle to bayes_results\obj_PDAC_outliers.pkl
-------------------------------
Outliers=False
-------------------------------
Already ran
Total fit time: 3.71 minutes


../bayes-drt\stan_models.py:7: UserWarning: Pickling fit objects is an experimental feature!
The relevant StanModel instance must be pickled along with this fit object.
When unpickling the StanModel must be unpickled first.
  pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)
